# Entrega 2 — Fast Prompting en Acción
**Curso: IA — Generación de Prompts**  
**Autor: Camilo Gómez**

---

## Introducción
- **Problema**: Los docentes y estudiantes suelen carecer de tiempo para crear material didáctico personalizado y atractivo. Esto impacta en la motivación y el rendimiento
- **Propuesta**: Generar cuestionarios, resúmenes y explicaciones paso a paso mediante prompts optimizados y el uso de la API de Gemini
- **Viabilidad**: Uso de Gemini (accesible), alcance acotado y costos bajos gracias a cache y parametrización


## Objetivos
1. Aplicar técnicas de **Fast Prompting**
2. Implementar una **POC en Jupyter Notebook**
3. Minimizar las llamadas a la API y registrar métricas
4. Probar diferentes tipos de materiales educativos: **quiz, resumen, explicación, prompts de imagen**


In [2]:
import os
from getpass import getpass
from dotenv import load_dotenv
import google.generativeai as genai


load_dotenv()

API_KEY = os.getenv("GEMINI_API_KEY")


if not API_KEY:
    API_KEY = getpass("Ingresá tu apikey ")

genai.configure(api_key=API_KEY)

print("Gemini configurado correctamente")


Ingresá tu apikey  ········


Gemini configurado correctamente


In [3]:
DEFAULT_MODEL = "gemini-1.5-flash"

GENERATION_CONFIG = {
    "temperature": 0.6,
    "top_p": 0.9,
    "top_k": 40,
    "max_output_tokens": 800
}

model = genai.GenerativeModel(
    model_name=DEFAULT_MODEL,
    generation_config=GENERATION_CONFIG
)


In [4]:
def quiz_prompt(topic, level="secundaria", n=5):
    return f"""
TAREA: Generar cuestionario de autoevaluación
CONTEXTO:
- Tema: {topic}
- Nivel: {level}

REQUISITOS:
- {n} preguntas variadas (múltiple opción o verdadero/falso).
- Incluye la respuesta correcta y una breve justificación.
- Salida en formato JSON válido.

SALIDA (ejemplo):
{{
  "topic": "{topic}",
  "questions": [
    {{"q": "Pregunta 1", "type":"multiple", "options":["a","b"], "answer":"a","why":"..." }}
  ]
}}
"""

def summary_prompt(topic, length="medio", audience="secundaria"):
    return f"""
TAREA: Resumir contenido educativo
CONTEXTO:
- Tema: {topic}
- Audiencia: {audience}
- Longitud: {length}

REQUISITOS:
- Resume en 3-5 ideas clave.
- Incluye un ejemplo práctico.
"""

def explanation_prompt(topic, steps=5):
    return f"""
TAREA: Explicación paso a paso
Tema: {topic}
Pasos: {steps}

REQUISITOS:
- Explica en {steps} pasos claros.
- Incluye al final 3 errores comunes y cómo evitarlos.
"""

def image_prompt(goal, style="educativo, claro, minimalista"):
    return f"""
Genera un prompt para NightCafe u otro generador de imágenes que represente:
"{goal}"
Estilo: {style}.
Debe incluir etiquetas claras y fondo limpio.
"""


In [5]:
def generate(prompt, task="general", model_name=DEFAULT_MODEL):
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(prompt)
    return response.text


In [6]:

quiz_text = generate(quiz_prompt("Fotosíntesis"), task="quiz")
print(quiz_text)

print(generate(summary_prompt("Revolución Francesa")))

print(generate(explanation_prompt("Derivadas básicas", steps=5)))

print(generate(image_prompt("El ciclo del agua para primaria")))


```json
{
  "topic": "Fotosíntesis",
  "questions": [
    {
      "q": "¿Cuál es el principal pigmento involucrado en la fotosíntesis?",
      "type": "multiple",
      "options": ["a) Clorofila", "b) Carotenoides", "c) Xantofilas", "d) Ficobilinas"],
      "answer": "a",
      "why": "La clorofila es el pigmento principal que absorbe la luz solar necesaria para la fotosíntesis. Los carotenoides y xantofilas son pigmentos accesorios."
    },
    {
      "q": "La fotosíntesis ocurre en...",
      "type": "multiple",
      "options": ["a) El núcleo celular", "b) Las mitocondrias", "c) Los cloroplastos", "d) El citoplasma"],
      "answer": "c",
      "why": "Los cloroplastos son los orgánulos celulares donde se lleva a cabo la fotosíntesis."
    },
    {
      "q": "Verdadero o Falso: La fotosíntesis es un proceso que libera oxígeno como subproducto.",
      "type": "boolean",
      "answer": true,
      "why": "Durante la fase luminosa de la fotosíntesis, se produce la fotólisis del agu

In [14]:
# Prompt naive
print("### Prompt naive")
print(generate("Explica la fotosíntesis"))

#  Fast Prompting
print("\n###Fast Prompting")
print(generate("""
Actúa como profesor de biología de secundaria.
Explica la fotosíntesis en 5 pasos claros y numerados.
Incluye un ejemplo práctico de la vida diaria.
Al final, menciona 3 errores comunes de los estudiantes y cómo evitarlos.
"""))


### Prompt naive
La fotosíntesis es el proceso mediante el cual las plantas, algas y algunas bacterias convierten la energía luminosa (principalmente la luz solar) en energía química en forma de glucosa (un azúcar).  Este proceso es fundamental para la vida en la Tierra, ya que es la base de la mayoría de las cadenas tróficas y proporciona el oxígeno que respiramos.

Se puede dividir en dos etapas principales:

**1. Fase luminosa (o fase dependiente de la luz):** Esta fase ocurre en las membranas de los tilacoides, estructuras internas de los cloroplastos (orgánulos celulares donde se realiza la fotosíntesis).  Aquí suceden los siguientes eventos:

* **Absorción de luz:** Los pigmentos fotosintéticos, principalmente la clorofila (verde), absorben la energía luminosa.  La clorofila a es el pigmento principal, mientras que otros pigmentos como la clorofila b y los carotenoides amplían el rango de longitudes de onda absorbidas.
* **Fotólisis del agua:** La energía absorbida se utiliza par

### Observación
El prompt naive devuelve una explicación general, a veces incompleta o muy técnica
El prompt optimizado con Fast Prompting en cambio estructura la respuesta en pasos, incluye ejemplos prácticos y errores comunes, lo que lo hace más útil para un contexto educativo

## Evaluación de costos y conclusiones
- Uso de **gemini-1.5-flash** para iteraciones rápidas
- Control de parámetros y uso de **prompts cortos** evita costos innecesarios
- Posible mejora: implementar **cache** para reusar respuestas idénticas
- Conclusión: el Fast Prompting permitió generar quizzes, resúmenes y explicaciones de manera rápida, estructurada y optimizada